In [2]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import pandas as pd
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

In [3]:
# MySQL 서버 연결 정보 설정
host = '15.164.171.183'
user = 'mealddang'
password = 'test1234'
database = 'mealddang'

# MySQL에 연결하는 Connection String 생성
connection_string = f'mysql+pymysql://{user}:{password}@{host}/{database}'

# 엔진 생성
engine = create_engine(connection_string)

In [4]:
# 쿼리 실행 및 결과 DataFrame으로 로드 Naver
query = "SELECT * FROM md_store_master;"
df = pd.read_sql(query, engine)
rest_list = df['store_id_n'].to_list()

In [5]:
head = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cache-Control": "max-age=0",
    "Cookie": "NNB=SUN7YEEJIPSWK; _fwb=152oJv7s0FXg11BB2jm0qlU.1710124682841; wcs_bt=sp_eddde133465c40:1710124757",
    "Sec-Ch-Ua": "\"Chromium\";v=\"122\", \"Not(A:Brand\";v=\"24\", \"Google Chrome\";v=\"122\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
}

In [32]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, inspect

# MySQL 서버 연결 정보 설정
host = '15.164.171.183'
user = 'mealddang'
password = 'test1234'
database = 'mealddang'

# MySQL에 연결하는 Connection String 생성
connection_string = f'mysql+pymysql://{user}:{password}@{host}/{database}'

# 엔진 생성
engine = create_engine(connection_string)

# metadata 생성
metadata = MetaData()

# 테이블 정의 (테이블의 이름, 컬럼 이름 및 데이터 타입 지정)
table_name = 'restaurant_keyword'  # 테이블 이름을 지정해야 합니다.
your_table = Table(
    table_name, metadata,
    Column('restaurant_id', Integer),
    Column('keyword', String(255))  # VARCHAR의 길이를 지정합니다.
)

# Inspector 객체 생성
inspector = inspect(engine)

# 테이블이 이미 존재하는지 확인하고 없으면 생성
if not inspector.has_table(table_name):
    metadata.create_all(engine)

In [33]:
# 각각의 값들을 MySQL 데이터베이스에 삽입하는 쿼리 작성 및 실행
for i in tqdm(range(len(rest_list))):
    url = f"https://m.place.naver.com/restaurant/{rest_list[i]}/menu/list"
    naver_r = requests.get(url, headers=head).content.decode('utf-8')
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(naver_r, 'html.parser')
    # 클래스가 "lPzHi"인 모든 요소 가져오기
    elements_with_class_lPzHi = soup.find_all(class_="lPzHi")
    # 각 요소의 텍스트 가져오기
    contents = [element.text for element in elements_with_class_lPzHi]

    for j in range(len(contents)):
        # Connection 객체 생성
        connection = engine.connect()
        # 삽입 쿼리 작성
        ins = your_table.insert().values(restaurant_id=rest_list[i], keyword=contents[j])
        
        
        # 쿼리 실행 및 커밋
        with connection.begin() as trans:
            connection.execute(ins)
            trans.commit()

100%|██████████| 1994/1994 [45:06<00:00,  1.36s/it] 
